In [1]:
!pip install catboost

In [2]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report

In [3]:
sentiment = pd.read_csv('/kaggle/input/gagarin-hack-data/sentiment.csv')
sentiment.drop('Unnamed: 0',axis = 1,inplace = True)
texts = pd.read_pickle('/kaggle/input/gagarin-hack-data/sentiment_texts.pickle')


In [4]:
texts.groupby('IsForward').agg('count')

,MessageID,ChannelID,issuerid,SentimentScore,DateAdded,DatePosted,MessageText
IsForward,,,,,,,
False,8676,8676,8676,8676,8676,8676,8676
True,613,613,613,613,613,613,613


In [5]:
texts

,MessageID,ChannelID,issuerid,SentimentScore,DateAdded,DatePosted,MessageText,IsForward
0,241407,1203560567,153,2,2023-05-12 19:03:20,2023-05-12 19:02:42,⚠️🇷🇺#SELG #дивиденд сд Селигдар: дивиденды 20...,False
1,33684,1136626166,230,4,2023-02-03 20:56:29,2023-02-03 16:46:34,Ozon продолжает развивать специализированные ф...,False
2,10090,1063908560,118,4,2023-06-02 19:18:37,2023-06-02 18:50:00,​Фокусы продолжаются🔥Акции и инвестиции 📈ВТБ ...,False
3,10090,1063908560,220,5,2023-06-02 19:18:37,2023-06-02 18:50:00,​Фокусы продолжаются🔥Акции и инвестиции 📈ВТБ ...,False
4,9826,1063908560,89,2,2023-04-24 17:51:38,2023-04-24 13:54:00,​​Windfall Tax — налог на сверхприбыль. Какие ...,False
...,...,...,...,...,...,...,...,...
9284,47482,1197210433,157,4,2023-03-20 14:53:14,2023-03-20 12:15:21,#FLOT #Дивиденды 💰 7% — возможная дивдоходност...,False
9285,233829,1203560567,157,4,2023-03-20 14:58:04,2023-03-20 12:05:49,🇷🇺#FLOT #отчетность ЧИСТАЯ ПРИБЫЛЬ СОВКОМФЛОТ...,False
9286,9789,1063908560,225,3,2023-04-19 17:51:56,2023-04-19 15:32:00,​​Ключевой принцип создания портфеля 🔹Диверси...,False
9287,233867,1203560567,127,3,2023-03-20 14:58:04,2023-03-20 14:33:32,"""💥🇷🇺#PLZL #листинг #торги """"Полюс"""" ведет диа...",False


In [6]:
sentiment

,MessageID,ChannelID,issuerid,SentimentScore
0,241407,1203560567,153,2
1,33684,1136626166,230,4
2,10090,1063908560,118,4
3,9826,1063908560,89,2
4,9672,1063908560,160,2
...,...,...,...,...
10926,9789,1063908560,225,3
10927,4789,1239405989,127,3
10928,233867,1203560567,127,3
10929,3681,1219484576,127,3


In [7]:
fin_df = sentiment.merge(texts,how ='right' ,on = 'MessageID')

In [8]:
sentiment.shape,texts.shape

((10931, 4), (9289, 8))

In [9]:
fin_df = fin_df.drop_duplicates(subset=['MessageID', 'issuerid_x'], keep='first')

In [10]:
fin_df.drop(['ChannelID_y','issuerid_y','SentimentScore_y'],axis =1,inplace = True)

In [11]:
fin_df

,MessageID,ChannelID_x,issuerid_x,SentimentScore_x,DateAdded,DatePosted,MessageText,IsForward
0,241407,1203560567,153,2,2023-05-12 19:03:20,2023-05-12 19:02:42,⚠️🇷🇺#SELG #дивиденд сд Селигдар: дивиденды 20...,False
1,33684,1136626166,230,4,2023-02-03 20:56:29,2023-02-03 16:46:34,Ozon продолжает развивать специализированные ф...,False
2,10090,1063908560,118,4,2023-06-02 19:18:37,2023-06-02 18:50:00,​Фокусы продолжаются🔥Акции и инвестиции 📈ВТБ ...,False
3,10090,1063908560,220,5,2023-06-02 19:18:37,2023-06-02 18:50:00,​Фокусы продолжаются🔥Акции и инвестиции 📈ВТБ ...,False
6,9826,1063908560,89,2,2023-04-24 17:51:38,2023-04-24 13:54:00,​​Windfall Tax — налог на сверхприбыль. Какие ...,False
...,...,...,...,...,...,...,...,...
22187,47482,1197210433,157,4,2023-03-20 14:53:14,2023-03-20 12:15:21,#FLOT #Дивиденды 💰 7% — возможная дивдоходност...,False
22188,233829,1203560567,157,4,2023-03-20 14:58:04,2023-03-20 12:05:49,🇷🇺#FLOT #отчетность ЧИСТАЯ ПРИБЫЛЬ СОВКОМФЛОТ...,False
22189,9789,1063908560,225,3,2023-04-19 17:51:56,2023-04-19 15:32:00,​​Ключевой принцип создания портфеля 🔹Диверси...,False
22190,233867,1203560567,127,3,2023-03-20 14:58:04,2023-03-20 14:33:32,"""💥🇷🇺#PLZL #листинг #торги """"Полюс"""" ведет диа...",False


In [12]:
dfs = np.array_split(fin_df, 10)

/opt/conda/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [20]:
import torch
from transformers import DistilBertTokenizer, DistilBertModel
from transformers import RobertaTokenizer, RobertaModel

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = RobertaTokenizer.from_pretrained('roberta-base', num_labels=5)
model = RobertaModel.from_pretrained('roberta-base', num_labels=5)

model.to(device)

def get_name_embedding(name, model, tokenizer):
    inputs = tokenizer(name, return_tensors='pt', padding=True, truncation=True).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    return torch.mean(outputs.last_hidden_state[:, 1:, :], dim=1).cpu().numpy()

fin_df['embedding'] = fin_df['MessageText'].apply(lambda x: get_name_embedding(x, model, tokenizer))
fin_df.head(10)

,MessageID,ChannelID_x,issuerid_x,SentimentScore_x,DateAdded,DatePosted,MessageText,IsForward,embedding
0,241407,1203560567,153,2,2023-05-12 19:03:20,2023-05-12 19:02:42,⚠️🇷🇺#SELG #дивиденд сд Селигдар: дивиденды 20...,False,"[[-0.04124578, -0.021663556, 0.03983897, 0.002..."
1,33684,1136626166,230,4,2023-02-03 20:56:29,2023-02-03 16:46:34,Ozon продолжает развивать специализированные ф...,False,"[[-0.064157285, 0.020365449, 0.035196174, 0.06..."
2,10090,1063908560,118,4,2023-06-02 19:18:37,2023-06-02 18:50:00,​Фокусы продолжаются🔥Акции и инвестиции 📈ВТБ ...,False,"[[-0.06888591, 0.0076898946, 0.030859321, 0.06..."
3,10090,1063908560,220,5,2023-06-02 19:18:37,2023-06-02 18:50:00,​Фокусы продолжаются🔥Акции и инвестиции 📈ВТБ ...,False,"[[-0.06888591, 0.0076898946, 0.030859321, 0.06..."
6,9826,1063908560,89,2,2023-04-24 17:51:38,2023-04-24 13:54:00,​​Windfall Tax — налог на сверхприбыль. Какие ...,False,"[[-0.060324408, 0.03924061, 0.03069084, 0.0641..."
7,9826,1063908560,127,2,2023-04-24 17:51:38,2023-04-24 13:54:00,​​Windfall Tax — налог на сверхприбыль. Какие ...,False,"[[-0.060324408, 0.03924061, 0.03069084, 0.0641..."
8,9826,1063908560,150,2,2023-04-24 17:51:38,2023-04-24 13:54:00,​​Windfall Tax — налог на сверхприбыль. Какие ...,False,"[[-0.060324408, 0.03924061, 0.03069084, 0.0641..."
9,9826,1063908560,7,2,2023-04-24 17:51:38,2023-04-24 13:54:00,​​Windfall Tax — налог на сверхприбыль. Какие ...,False,"[[-0.060324408, 0.03924061, 0.03069084, 0.0641..."
22,9672,1063908560,160,2,2023-04-04 14:44:28,2023-04-04 11:35:09,"У нас было 2 пакета префов Сургутнефтегаза, 75...",False,"[[-0.050761968, 0.028457744, 0.032483656, 0.08..."
23,9694,1063908560,254,4,2023-04-06 20:44:23,2023-04-06 18:54:08,"""​​""""Бесплатных"""" долларов больше нет🔥Акции и ...",False,"[[-0.065581806, 0.026415423, 0.01773477, 0.080..."


In [14]:
sas = fin_df[fin_df['SentimentScore_x'] == 0]

In [15]:

fin_df = fin_df[~fin_df['MessageID'].isin(sas['MessageID'])]


In [16]:
fin_df

,MessageID,ChannelID_x,issuerid_x,SentimentScore_x,DateAdded,DatePosted,MessageText,IsForward,embedding
0,241407,1203560567,153,2,2023-05-12 19:03:20,2023-05-12 19:02:42,⚠️🇷🇺#SELG #дивиденд сд Селигдар: дивиденды 20...,False,"[[-0.04124578, -0.021663556, 0.03983897, 0.002..."
1,33684,1136626166,230,4,2023-02-03 20:56:29,2023-02-03 16:46:34,Ozon продолжает развивать специализированные ф...,False,"[[-0.064157285, 0.020365449, 0.035196174, 0.06..."
2,10090,1063908560,118,4,2023-06-02 19:18:37,2023-06-02 18:50:00,​Фокусы продолжаются🔥Акции и инвестиции 📈ВТБ ...,False,"[[-0.06888591, 0.0076898946, 0.030859321, 0.06..."
3,10090,1063908560,220,5,2023-06-02 19:18:37,2023-06-02 18:50:00,​Фокусы продолжаются🔥Акции и инвестиции 📈ВТБ ...,False,"[[-0.06888591, 0.0076898946, 0.030859321, 0.06..."
6,9826,1063908560,89,2,2023-04-24 17:51:38,2023-04-24 13:54:00,​​Windfall Tax — налог на сверхприбыль. Какие ...,False,"[[-0.060324408, 0.03924061, 0.03069084, 0.0641..."
...,...,...,...,...,...,...,...,...,...
22187,47482,1197210433,157,4,2023-03-20 14:53:14,2023-03-20 12:15:21,#FLOT #Дивиденды 💰 7% — возможная дивдоходност...,False,"[[-0.07165859, 0.01855125, 0.030204114, 0.0780..."
22188,233829,1203560567,157,4,2023-03-20 14:58:04,2023-03-20 12:05:49,🇷🇺#FLOT #отчетность ЧИСТАЯ ПРИБЫЛЬ СОВКОМФЛОТ...,False,"[[-0.053833634, 0.0035359743, 0.03872551, 0.07..."
22189,9789,1063908560,225,3,2023-04-19 17:51:56,2023-04-19 15:32:00,​​Ключевой принцип создания портфеля 🔹Диверси...,False,"[[-0.06937917, 0.030827511, 0.023592308, 0.063..."
22190,233867,1203560567,127,3,2023-03-20 14:58:04,2023-03-20 14:33:32,"""💥🇷🇺#PLZL #листинг #торги """"Полюс"""" ведет диа...",False,"[[-0.08738015, -0.001989541, 0.049447462, 0.03..."


In [17]:
print(

SyntaxError: incomplete input (149104261.py, line 1)

In [ ]:
# X = np.vstack(fin_df['embedding'].values)
# y = fin_df['SentimentScore_x']

In [ ]:
# from sklearn.utils import compute_class_weight

# class_weights = compute_class_weight('balanced', classes=np.unique(fin_df['SentimentScore_x']), y=fin_df['SentimentScore_x'])
# class_weights

In [ ]:
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=666)

# train_pool = Pool(X_train, label=y_train)
# test_pool = Pool(X_val, label=y_val)


# models = [CatBoostClassifier(iterations=14000, learning_rate=0.0075, depth=d, loss_function='MultiClass') for d in [3,4,5,6,7]]
# for index,model in enumerate(models):
#     model.fit(train_pool, eval_set=test_pool, early_stopping_rounds=300,verbose = 1000)
#     filename = f'catboost_model_{index}.cbm' 
#     model.save_model(filename)

# import numpy as np
# from scipy.stats import mode
# predictions = [model.predict(X_val) for model in models]
# predictions = np.array(predictions)
# predictions_array = np.column_stack(predictions)
# hard_voting = []
# for i in range(predictions_array.shape[0]):
#     hard_voting.append(mode(predictions_array[i]).mode)
# hard_voting = np.array(hard_voting)

    

In [ ]:
# import numpy as np
# from scipy.stats import mode
# models=[]
# for i in range(4):
#     model = CatBoostClassifier()
#     model.load_model(f"/kaggle/working/catboost_model_{i}.cbm")
#     models.append(model)

In [ ]:
# predictions = [model.predict(X_val) for model in models]
# predictions = np.array(predictions)
# predictions_array = np.column_stack(predictions)
# hard_voting = []
# for i in range(predictions_array.shape[0]):
#     hard_voting.append(mode(predictions_array[i]).mode)
# hard_voting = np.array(hard_voting)


In [ ]:
# from sklearn.metrics import accuracy_score
# accuracy = accuracy_score(y_val, hard_voting)
# print(f'Accuracy with Hard Voting: {accuracy:.4f}')
# with open('example1.txt', 'w') as file:
#     file.write(str(accuracy))


In [ ]:
# for index, model in enumerate(models):
#     filename = f'catboost_model_{index}.cbm' 
#     model.save_model(filename)
#     print(f'Model {index} saved as {filename}')


In [ ]:
# hard_voting.shape

In [ ]:

#     file.write(val_accuracy)


In [ ]:
# # Use a pipeline as a high-level helper
# from transformers import pipeline
# pipe = pipeline("text-classification", model="seara/rubert-tiny2-russian-sentiment")

In [ ]:
# from transformers import pipeline

# # pipe = pipeline("text-classification", model="sismetanin/mbart_ru_sum_gazeta-ru-sentiment-rusentiment")ф

In [ ]:
# pipe("Привет, я твоего друга очень люблю!")

In [ ]:
# fin_df